## Differential Equation Solving First Notebook

In [1]:
using Plots

 Initialization of the grid and the function we want to solve numerically. Here we solve $\phi_{'t}=\phi_{'x}$.

In [79]:
function initial(Nx::Int)
    x=zeros(Nx)
    h=10/(Nx-1)
    for i in 1:Nx
        x[i] = (i-1)*h
    end
    un=zeros(Nx)
    uo=zeros(Nx)
    ##initialize
    for i in 1:Nx
        uo[i] = exp(-(x[i]-5)^2) - exp(-25)
    end
    return uo, un, x, h
end;

function initialSin(Nx::Int)
   x=zeros(Nx)
    h=2*pi/(Nx-1)
    for i in 1:Nx
        x[i] = (i-1)*h
    end
    un=zeros(Nx)
    uo=zeros(Nx)
    ##initialize
    for i in 1:Nx
        uo[i] = sin(x[i])
    end
    return uo, un, x, h 
end;

In [22]:
initial(100)

([0.0, 2.38595e-11, 8.66375e-11, 2.48414e-10, 6.56715e-10, 1.66595e-9, 4.10904e-9, 9.90087e-9, 2.33474e-8, 5.39184e-8  …  5.39184e-8, 2.33474e-8, 9.90087e-9, 4.10904e-9, 1.66595e-9, 6.56715e-10, 2.48414e-10, 8.66375e-11, 2.38595e-11, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.10101, 0.20202, 0.30303, 0.40404, 0.505051, 0.606061, 0.707071, 0.808081, 0.909091  …  9.09091, 9.19192, 9.29293, 9.39394, 9.49495, 9.59596, 9.69697, 9.79798, 9.89899, 10.0], 0.10101010101010101)

The orders of the derivative operater that are implemented.

In [3]:
function centeredDeriv(iter, lam, u)
    if iter in 2:length(u)-1
        return lam*(u[iter+1]-u[iter-1])/2
    else
        return 0
    end
end

function rightDerivFor(iter, lam, u)
    if iter in 1:length(u)-1
        return lam*(u[iter+1]-u[iter])
    else
        return 0
    end
end;

function rightDerivBack(iter, lam, u)
    if iter in 2:length(u)
        return lam*(u[iter]-u[iter-1])
    else
        return 0
    end
end;

In [35]:
function evaluate(Nx::Int, Nt::Int, lambda::Float64, deriv::Function, filename::String, fpss::Int)
    uo, un, x = initial(Nx)
    ##update
    anim = @animate for j in 1:Nt
        for i in 1:Nx
            un[i] = uo[i] + deriv(i, lambda, uo)
        end
        if deriv == rightDerivFor
            un[Nx] = un[1]
            elseif deriv == rightDerivBack
            un[1] = un[Nx]
        end
        
        for i in 1:Nx
            uo[i] = un[i]
        end
    plot(x, un, ylims = (0,1), xlims = (x[1],x[end]), leg=false)
    end
    fil = "C:/Users/Christopher/Application Data/Local/Temp/"*filename*".gif" 
    return gif(anim, fil, fps=fpss)
end;

In [37]:
evaluate(501, 100, -1., rightDerivBack, "GaussFirst", 30)

┌ Info: Saved animation to 
│   fn = C:\Users\Christopher\Application Data\Local\Temp\GaussFirst.gif
└ @ Plots C:\Users\Christopher\.julia\packages\Plots\rmogG\src\animation.jl:90


Plots.AnimatedGif("C:\\Users\\Christopher\\Application Data\\Local\\Temp\\GaussFirst.gif")

In [5]:
function evaluateBoundary(Nx::Int, Nt::Int, lambda::Float64, deriv::Function, filename::String, fpss::Int)
    uo, un, x = initial(Nx)
    ##update
    anim = @animate for j in 1:Nt
        for i in 1:Nx-1
            un[i] = uo[i] + deriv(i, lambda, uo)
        end
        un[Nx] = 0
        for i in 1:Nx
            uo[i] = un[i]
        end
    plot(x, un, ylims = (0,1), xlims = (x[1],x[end]), leg=false)
    end
    fil = "C:/Users/Christopher/Application Data/Local/Temp/"*filename*".gif" 
    return gif(anim, fil, fps=fpss)
end;

In [17]:
evaluateBoundary(501, 600, 1., rightDeriv, "rightBound", 30)

┌ Info: Saved animation to 
│   fn = C:\Users\Christopher\Application Data\Local\Temp\rightBound.gif
└ @ Plots C:\Users\Christopher\.julia\packages\Plots\rmogG\src\animation.jl:90


Plots.AnimatedGif("C:\\Users\\Christopher\\Application Data\\Local\\Temp\\rightBound.gif")

In [4]:
function evaluateNoGIF(Nx::Int, Nt::Int, lambda::Float64, deriv::Function)
    uo, un, x = initial(Nx)
    store = zeros(Nt, Nx)
    ##update
    for j in 1:Nt
        for i in 1:Nx-1
            un[i] = uo[i] + deriv(i, lambda, uo)
        end
        un[Nx] = un[1]
        for i in 1:Nx
            uo[i] = un[i]
        end
        store[j,:] = un
    end
    return store
end;

In [67]:
evaluateNoGIF(501, 200, 1., rightDerivFor);

In [5]:
function convergence(Nx::Int, Nt::Int, lambda::Float64, evaluateF::Function)
    n=zeros(Nx)
    d=zeros(Nx)
    results = zeros(Nt)
    if evaluateF == evaluateNoGIF
        zv1 = evaluateF(Nx, Nt, lambda, rightDerivFor)
        zv2 = evaluateF(2*Nx-1, Nt, lambda, rightDerivFor)
        zv3 = evaluateF(4*Nx-3, Nt, lambda, rightDerivFor)
    end
    if evaluateF == evaluateWaveSecond
        zv1 = evaluateF(Nx,Nt, lambda)[1]
        zv2 = evaluateF(2*Nx-1,Nt, lambda)[1]
        zv3 = evaluateF(4*Nx-3,Nt, lambda)[1]
    end
    for i in 1:Nt
        for j in 1:Nx #since h=10/(Nx-1), we see h/2=10/((2Nx-1)-1) and h/4=10/((4*Nx-3)-1)
            n[j] = (zv1[i,j]-zv2[i,2*j-1])^2
            d[j] = (zv2[i,2*j-1]-zv3[i,4*j-3])^2
        end
        results[i] = sqrt(sum(n))/sqrt(sum(d))
    end
    return sum(results)/Nt
end;

In [100]:
print("2^q = ", convergence(2001, 100, 1., evaluateNoGIF))

2^q = 1.961793597129909

## Next wave equation

Here I try to solve $\phi_{'tt}=\phi_{'xx}$. First, let's introduce $f = \phi_{'t}$ and $g = \phi_{'x}$. Therefore we get $f_{'t} = g_{'x}$ as well as $g_{'t} = f_{'x}$. We use a forward derivative for the time steps. 
Therefore, we get (for timestep n, index cell J):

\begin{align*}
\left. f_{'t} \right|_J^n &= \frac{g_{J+1}^n-g_{J-1}^n}{2 \Delta x} \\
\left. g_{'t} \right|_J^n &= \frac{f_{J+1}^n-f_{J-1}^n}{2 \Delta x} \\
\Rightarrow f_J^{n+1} &= f_J^n + \frac{\Delta t}{2\Delta x} (g_{J+1}^n-g_{J-1}^n) \\
\Rightarrow g_J^{n+1} &= g_J^n + \frac{\Delta t}{2\Delta x} (f_{J+1}^n-f_{J-1}^n) \\ \\
\phi_J^{n+1} &= \phi_J^n + \Delta t \cdot f_J^n = \phi_J^n + \Delta t \cdot f_J^{n-1} + \Delta t^2 \cdot \frac{g_{J+1}^{n-1}-g_{J-1}^{n-1}}{2 \Delta x}
\end{align*}

In [60]:
#Unstable
function evaluateWave(Nx::Int, Nt::Int, lambda::Float64)
    store = zeros(Nt, Nx)
    #declaration of the variables
    go, gn, po, pn, fo, goo = zeros(Nx), zeros(Nx), zeros(Nx), zeros(Nx), initial(Nx)[1], zeros(Nx)
    #initialization
    foo, fn, x, deltax = initial(Nx)
    #go[2] = go[1]
    #go[end-1]=go[end]
    deltat = lambda*deltax
    # only for guassian @. go = -2*(x-5)*exp(-(x-5)^2)
    ##update
    for j in 1:Nt # anim = @animate
        #update f and g
        for i in 2:Nx-1
            fn[i] = fo[i] + centeredDeriv(i, lambda, go)
            gn[i] = go[i] + centeredDeriv(i, lambda, fo)
        end
        #HERE: one-sided derivatives at boundaries
        #fn[1] = fo[1] + lambda*(go[2]-go[1])
        #fn[end] = fo[end] + lambda*(go[end]-go[end-1])
        #gn[1] = go[1] + lambda*(fo[2]-fo[1])
        #gn[end] = go[end] + lambda*(fo[end]-fo[end-1])
        #boundaries: p[1] = p[end] = 0
        #fn[2] = fn[1]
        #fn[end-1]=fn[end]  
        #gn[1] = 0
        #gn[end] = 0
        #update phi
        for i in 2:Nx-1
            pn[i] = po[i] + deltat*fo[i]
        end
        pn[1] = po[1] + deltat*foo[1] + deltat^2/(deltax)*(goo[2]-goo[1])
        pn[end] = po[end] + deltat*foo[end] + deltat^2/(deltax)*(goo[end]-goo[end-1])
        #println(go[2], ", ", go[end-1], ", ",pn[2], ", ", pn[end-1]) #debug
        #shift new values to old
        foo .= fo
        goo .= go
        fo .= fn
        go .= gn
        po .= pn
        store[j,:] = pn
    end
    return store, x
end;

In [61]:
evaluateWave(501, 300, .9)

(getfield(Main, Symbol("#store#8")){Array{Float64,1}}([-4.41921e20, 6.64296e21, 2.62329e21, -1.95959e22, -5.1151e21, 3.15548e22, 7.34582e21, -4.18763e22, -9.19108e21, 4.99586e22  …  4.99586e22, -9.19108e21, -4.18763e22, 7.34582e21, 3.15548e22, -5.1151e21, -1.95959e22, 2.62329e21, 6.64296e21, -4.41921e20]), [0.0, 0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 0.14, 0.16, 0.18  …  9.82, 9.84, 9.86, 9.88, 9.9, 9.92, 9.94, 9.96, 9.98, 10.0])

Here I try an implicit Euler solution. Again, $f = \phi_{'t}$ and $g = \phi_{'x}$. However, $f$ and $g$ still are calculated explicitely.

In [47]:
function evaluateWaveImplicit(Nx::Int, Nt::Int, lambda::Float64)
    store = zeros(Nt, Nx)
    #declaration of the variables
    fo, fn, go, gn = zeros(Nx), zeros(Nx), zeros(Nx), zeros(Nx)
    #initialization
    po, pn, x, deltax = initial(Nx)
    deltat = lambda*deltax
    fo = initial(Nx)[1]
    #@. go = -2*abs(x-5)*exp(-(x-5)^2) + 10 * exp(-25)
    ##update
    for j in 1:Nt #anim = @animate 
        #update f and g with FTCS
        for i in 2:Nx-1
            fn[i] = fo[i] + centeredDeriv(i, lambda, go)
            gn[i] = go[i] + centeredDeriv(i, lambda, fo)
        end
        #HERE: one-sided derivatives at boundaries
        gn[1] = gn[2]
        gn[end] = gn[end-1]
        fn[1] = fo[1] + lambda*(go[2]-go[1])
        fn[end] = fo[end] + lambda*(go[end]-go[end-1])
        #boundaries: p[1] = p[end] = 0
        #fn[2] = fn[1]
        #fn[end-1]=fn[end]  
        #gn[1] = 0
        #gn[end] = 0
        #update phi
        @. pn = po + deltat*fn
        if pn[1] != 0 || pn[end]!=0
            println(pn[1], " ,", pn[end], " ;", j)
            break
        end
        #shift new values to old
        po .= pn
        fo .= fn
        go .= gn
        #println(go[1], ", ", go[end], ", ",pn[2], ", ", pn[end-1]) #debug
        store[j,:] = pn
    end
    return store, x
end;

In [59]:
fgSol = evaluateWaveImplicit(501, 200, .1)

([0.0 3.07418e-12 … 3.07418e-12 0.0; 0.0 3.08032e-12 … 3.08032e-12 0.0; … ; 0.0 1.45823e-11 … 1.45823e-11 0.0; 0.0 1.48004e-11 … 1.48004e-11 0.0], [0.0, 0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 0.14, 0.16, 0.18  …  9.82, 9.84, 9.86, 9.88, 9.9, 9.92, 9.94, 9.96, 9.98, 10.0])

In [46]:
createGIF(fgSol[1], fgSol[2], "testfg", 10)

┌ Info: Saved animation to 
│   fn = C:\Users\Christopher\Application Data\Local\Temp\testfg.gif
└ @ Plots C:\Users\Christopher\.julia\packages\Plots\rmogG\src\animation.jl:90


Plots.AnimatedGif("C:\\Users\\Christopher\\Application Data\\Local\\Temp\\testfg.gif")

# New Try with second derivative operator

Okay, completely new try because that does not work out the way I want to. I'll come back to this method later.

Here, I just try to implement a solution I found online that uses the second derivative operator. The function $evaluateWaveSecond$ generates a 2D array that stores the value of $\phi$ at each point in time.

In [62]:
function evaluateWaveSecond(Nx::Int, Nt::Int, lambda::Float64)
    store = zeros(Nt, Nx)
    #initialization
    po = zeros(Nx)
    poo, pn, x, deltax = initial(Nx)
    v0 = zeros(Nx)
    deltat = lambda*deltax
    for i in 2:Nx-1
        po[i] = 0.5*lambda^2*(poo[i-1]+poo[i+1])+(1-lambda^2)*poo[i]+deltat*v0[i] #version i have a calculation to
        #po[i] = poo[i]*lambda^2 + 0.5*(1-lambda^2) * (poo[i+1]+poo[i-1])+deltat*v0[i]
    end
    po[1] = 0
    po[end] = 0
    # only for guassian @. go = -2*(x-5)*exp(-(x-5)^2)
    ##update
    for j in 1:Nt 
        #update f and g with FTCS
        for i in 2:Nx-1
            pn[i] = -poo[i] + 2*po[i]*(1-lambda^2) + lambda^2 * (po[i+1]+po[i-1])
        end
        pn[1] = 0
        pn[end] = 0
        #shift new values to old
        plot(x, poo, ylims = (-1,1), xlims = (x[1],x[end]), leg=false)
        poo .= po
        po .= pn
        store[j,:] = pn
    end
    return store, x
    end;

In [64]:
sol = evaluateWaveSecond(501, 1000, 1.)

([0.0 4.12928e-12 … 4.12928e-12 0.0; 0.0 5.01288e-12 … 5.01288e-12 0.0; … ; 0.0 3.06799e-12 … 3.06799e-12 0.0; 0.0 3.39895e-12 … 3.39861e-12 0.0], [0.0, 0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 0.14, 0.16, 0.18  …  9.82, 9.84, 9.86, 9.88, 9.9, 9.92, 9.94, 9.96, 9.98, 10.0])

In [29]:
function createGIF(store::Array{Float64, 2}, x::Array{Float64, 1}, filename::String, fpss::Int)
    anim = @animate for i in 1:length(store[:,1])
        plot(x, store[i,:], ylims = (-1,1), xlims = (x[1],x[end]), leg=false)
    end
    fil = "C:/Users/Christopher/Application Data/Local/Temp/"*filename*".gif" 
    return gif(anim, fil, fps=fpss)
end;

In [44]:
createGIF(sol[1], sol[2], "test", 10)

┌ Info: Saved animation to 
│   fn = C:\Users\Christopher\Application Data\Local\Temp\test.gif
└ @ Plots C:\Users\Christopher\.julia\packages\Plots\rmogG\src\animation.jl:90


Plots.AnimatedGif("C:\\Users\\Christopher\\Application Data\\Local\\Temp\\test.gif")

In [99]:
convergence(10001, 10000, 1., evaluateWaveSecond)

1.1965998543097103

Ok so from now on I try to continue with the extended forumla $\phi_{'tt} = \phi_{'xx} - \beta^2 \phi$. 

In [88]:
function evaluateWaveSecondNew(Nx::Int, Nt::Int, lambda::Float64, beta::Float64)
    store = zeros(Nt, Nx)
    #initialization
    po = zeros(Nx)
    poo, pn, x, deltax = initial(Nx)
    v0 = zeros(Nx)
    deltat = lambda*deltax
    for i in 2:Nx-1
        po[i] = 0.5*lambda^2*(poo[i-1]+poo[i+1])+(1-lambda^2)*poo[i]+deltat*v0[i]-deltat^2*beta^2/2*poo[i]  
        #       as before                                                        from the added term    
    end
    po[1] = 0
    po[end] = 0
    ##update
    for j in 1:Nt 
        #update f and g with FTCS
        for i in 2:Nx-1
            pn[i] = -poo[i] + 2*po[i]*(1-lambda^2) + lambda^2 * (po[i+1]+po[i-1])-deltat^2*beta^2*po[i]
        end
        pn[1] = 0
        pn[end] = 0
        #shift new values to old        
        plot(x, poo, ylims = (-1,1), xlims = (x[1],x[end]), leg=false)
        poo .= po
        po .= pn
        store[j,:] = pn
    end
    return store, x
    end;

In [94]:
solNew = evaluateWaveSecondNew(51, 1000, .99, 1.)

([0.0 1.82713e-9 … 1.82713e-9 0.0; 0.0 9.93861e-9 … 9.93861e-9 0.0; … ; 0.0 0.134519 … 0.134519 0.0; 0.0 0.174464 … 0.174464 0.0], [0.0, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2, 1.4, 1.6, 1.8  …  8.2, 8.4, 8.6, 8.8, 9.0, 9.2, 9.4, 9.6, 9.8, 10.0])

In [95]:
createGIF(solNew[1], solNew[2], "test", 30)

┌ Info: Saved animation to 
│   fn = C:\Users\Christopher\Application Data\Local\Temp\test.gif
└ @ Plots C:\Users\Christopher\.julia\packages\Plots\rmogG\src\animation.jl:90


Plots.AnimatedGif("C:\\Users\\Christopher\\Application Data\\Local\\Temp\\test.gif")

In [80]:
function evaluateWaveSecondSin(Nx::Int, Nt::Int, lambda::Float64)
    store = zeros(Nt, Nx)
    #initialization
    po = zeros(Nx)
    poo, pn, x, deltax = initial(Nx)
    v0 = zeros(Nx)
    deltat = lambda*deltax
    for i in 2:Nx-1
        po[i] = 0.5*lambda^2*(poo[i-1]+poo[i+1])+(1-lambda^2)*poo[i]+deltat*v0[i] -deltat^2/2*sin(poo[i])   
    end
    po[1] = 0
    po[end] = 0
    ##update
    for j in 1:Nt 
        for i in 2:Nx-1
            pn[i] = -poo[i] + 2*po[i]*(1-lambda^2) + lambda^2 * (po[i+1]+po[i-1]) - deltat^2*sin(po[i])
        end
        pn[1] = 0
        pn[end] = 0
        #shift new values to old
        plot(x, poo, ylims = (-1,1), xlims = (x[1],x[end]), leg=false)
        poo .= po
        po .= pn
        store[j,:] = pn
    end
    return store, x
    end;

In [84]:
solSin = evaluateWaveSecondSin(101, 100, 1.)

([0.0 1.07935e-10 … 1.07935e-10 0.0; 0.0 2.70171e-10 … 2.70171e-10 0.0; … ; 0.0 -0.000906597 … -0.000906597 0.0; 0.0 0.00249345 … 0.00249345 0.0], [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9  …  9.1, 9.2, 9.3, 9.4, 9.5, 9.6, 9.7, 9.8, 9.9, 10.0])

In [81]:
createGIF(solSin[1], solSin[2], "test", 30)

┌ Info: Saved animation to 
│   fn = C:\Users\Christopher\Application Data\Local\Temp\test.gif
└ @ Plots C:\Users\Christopher\.julia\packages\Plots\rmogG\src\animation.jl:90


Plots.AnimatedGif("C:\\Users\\Christopher\\Application Data\\Local\\Temp\\test.gif")